In [1]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
import xgboost as xgb
import timeit
import optuna


In [1]:
import ttp_dataload
import ttp_data_cleaning
import ttp_data_transformations
import optuna_tuning

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = ttp_dataload.download_data()

df = ttp_data_cleaning.full_cleaning(df)

categorical_features = ['subject','reason','queue','department','fire_district',
                        'pwd_district','city_council_district','police_district',
                        'neighborhood','neighborhood_services_district','ward',
                        'source','open_dt_month', 'open_dt_week','open_dt_day',
                        'open_dt_hour','open_dt_dayofweek','closed_dt_month',
                        'closed_dt_week','closed_dt_day','closed_dt_hour',
                        'closed_dt_dayofweek']

data_processed = ttp_data_transformations.data_transformations(df, categorical_features)

data preprocessing started

column selection started

feature engineering started

start dataframe train validation trainvalidationsplit

data is good, we can work
start X y splits and preprocessing


### Lasso

In [4]:
optuna_tuning.linear_lasso(data_processed.x_train_processed, 
                           data_processed.x_valid_processed, 
                           data_processed.x_test_processed, 
                           data_processed.x_trainvalid_processed, 
                           data_processed.y_train, 
                           data_processed.y_valid, 
                           data_processed.y_test, 
                           data_processed.y_trainvalid, 1)

[I 2020-11-07 19:10:44,994] A new study created in memory with name: no-name-da0d18ec-7b1f-4011-806c-5587711f09b0
[I 2020-11-07 19:13:28,776] Trial 0 finished with value: 284066.6015909198 and parameters: {'lasso_alpha': 0.0017826497151386949}. Best is trial 0 with value: 284066.6015909198.


Time for preparing Lasso result:  7.73  minutes


{'model': 'lasso',
 'hyperparameters': [{'lasso_alpha': 0.0017826497151386949}],
 'mse_train': 1414167.0995753251,
 'mse_test': 402496.9407144272,
 'mse_validation': 284066.6015909198}

In [3]:
optuna_tuning.linear_ridge(data_processed.x_train_processed, 
                           data_processed.x_valid_processed, 
                           data_processed.x_test_processed, 
                           data_processed.x_trainvalid_processed, 
                           data_processed.y_train, 
                           data_processed.y_valid, 
                           data_processed.y_test, 
                           data_processed.y_trainvalid, 1)

[I 2020-11-07 19:10:27,224] A new study created in memory with name: no-name-e6aa9022-fdf6-47e5-97af-552fa0cfe42a
[I 2020-11-07 19:10:30,968] Trial 0 finished with value: 284777.8240425932 and parameters: {'ridge_alpha': 0.0017826497151386949}. Best is trial 0 with value: 284777.8240425932.


Time for preparing Ridge result:  0.17  minutes


{'model': 'ridge',
 'hyperparameters': [{'ridge_alpha': 0.0017826497151386949}],
 'mse_train': 1414072.6847331831,
 'mse_test': 403078.86275889684,
 'mse_validation': 284777.8240425932}

In [4]:
optuna_tuning.lightgbm_model(categorical_features,
                             data_processed.x_train, 
                             data_processed.x_valid, 
                             data_processed.x_test, 
                             data_processed.x_trainvalid, 
                             data_processed.y_train, 
                             data_processed.y_valid, 
                             data_processed.y_test, 
                             data_processed.y_trainvalid, 1)

[I 2020-11-07 19:23:00,508] A new study created in memory with name: no-name-af63a72b-c38f-4f87-9a86-39579f44ed86
[W 2020-11-07 19:23:00,510] Trial 0 failed because of the following error: TypeError('Wrong type(str) or unknown name(subject) in categorical_feature')
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/site-packages/optuna/_optimize.py", line 189, in _run_trial
    value = func(trial)
  File "/Users/denissai/MIT/FALL_2020/15.072_Advanced_Analytics_Edge/aae_project/optuna_tuning.py", line 142, in objective_lightgbm
    gbm = lgb.train(param, lgb_train)
  File "/usr/local/lib/python3.8/site-packages/lightgbm/engine.py", line 231, in train
    booster = Booster(params=params, train_set=train_set)
  File "/usr/local/lib/python3.8/site-packages/lightgbm/basic.py", line 1983, in __init__
    train_set.construct()
  File "/usr/local/lib/python3.8/site-packages/lightgbm/basic.py", line 1321, in construct
    self._lazy_init(self.data, label=self.label,
  File "/us

TypeError: Wrong type(str) or unknown name(subject) in categorical_feature

In [ ]:
optuna_tuning.xgboost_model(data_processed.x_train_processed, 
                             data_processed.x_valid_processed, 
                             data_processed.x_test_processed, 
                             data_processed.x_trainvalid_processed, 
                             data_processed.y_train, 
                             data_processed.y_valid, 
                             data_processed.y_test, 
                             data_processed.y_trainvalid, 1)

### CatBoost

In [37]:
def column_index(df, query_cols):
    cols = df.columns.values
    sidx = np.argsort(cols)
    return sidx[np.searchsorted(cols,query_cols,sorter=sidx)]

# cb_model = CatBoostRegressor(iterations=700,
#                              learning_rate=0.02,
#                              depth=12,
#                              eval_metric='RMSE',
#                              random_seed = 23,
#                              bagging_temperature = 0.2,
#                              od_type='Iter',
#                              metric_period = 75,
#                              od_wait=100)

cb_model = CatBoostRegressor(iterations = 200, eval_metric = 'RMSE')

cb_model.fit(X_train, y_train,
             eval_set=(X_valid,y_valid),
             cat_features=column_index(X_train,categorical_features_names),
             use_best_model=True,
             verbose=True)

Learning rate set to 0.388763
0:	learn: 1108.1341950	test: 502.8311893	best: 502.8311893 (0)	total: 304ms	remaining: 1m
1:	learn: 935.1006945	test: 373.2058241	best: 373.2058241 (1)	total: 611ms	remaining: 1m
2:	learn: 912.4845276	test: 377.3743232	best: 373.2058241 (1)	total: 864ms	remaining: 56.7s
3:	learn: 834.3310734	test: 336.6164973	best: 336.6164973 (3)	total: 1.08s	remaining: 52.8s
4:	learn: 807.1475838	test: 315.9880488	best: 315.9880488 (4)	total: 1.5s	remaining: 58.7s
5:	learn: 794.4703868	test: 309.9452103	best: 309.9452103 (5)	total: 1.93s	remaining: 1m 2s
6:	learn: 783.5616297	test: 314.0956189	best: 309.9452103 (5)	total: 2.43s	remaining: 1m 6s
7:	learn: 766.2329797	test: 303.5131550	best: 303.5131550 (7)	total: 2.74s	remaining: 1m 5s
8:	learn: 750.3991385	test: 296.1603334	best: 296.1603334 (8)	total: 3.01s	remaining: 1m 3s
9:	learn: 739.1660462	test: 301.1265678	best: 296.1603334 (8)	total: 3.38s	remaining: 1m 4s
10:	learn: 731.8629331	test: 297.6558465	best: 296.16033

88:	learn: 597.1482446	test: 322.2274714	best: 267.4440268 (36)	total: 35.8s	remaining: 44.7s
89:	learn: 596.7588595	test: 322.0349364	best: 267.4440268 (36)	total: 36.2s	remaining: 44.3s
90:	learn: 595.6217075	test: 321.0771234	best: 267.4440268 (36)	total: 36.6s	remaining: 43.8s
91:	learn: 594.4883183	test: 321.7262508	best: 267.4440268 (36)	total: 36.9s	remaining: 43.3s
92:	learn: 594.3064156	test: 321.5227188	best: 267.4440268 (36)	total: 37.3s	remaining: 42.9s
93:	learn: 594.2296221	test: 321.2805972	best: 267.4440268 (36)	total: 37.6s	remaining: 42.4s
94:	learn: 593.8214597	test: 321.3227474	best: 267.4440268 (36)	total: 38s	remaining: 42.1s
95:	learn: 593.1610077	test: 325.1804867	best: 267.4440268 (36)	total: 38.5s	remaining: 41.7s
96:	learn: 592.9938379	test: 325.3096229	best: 267.4440268 (36)	total: 38.9s	remaining: 41.3s
97:	learn: 592.9557450	test: 325.3193895	best: 267.4440268 (36)	total: 39.2s	remaining: 40.8s
98:	learn: 592.5859557	test: 324.4552813	best: 267.4440268 (36

175:	learn: 566.6037293	test: 469.9660103	best: 267.4440268 (36)	total: 1m 8s	remaining: 9.35s
176:	learn: 566.4771220	test: 470.0103837	best: 267.4440268 (36)	total: 1m 8s	remaining: 8.96s
177:	learn: 566.3042726	test: 470.2024099	best: 267.4440268 (36)	total: 1m 9s	remaining: 8.57s
178:	learn: 565.9618398	test: 470.5524393	best: 267.4440268 (36)	total: 1m 9s	remaining: 8.18s
179:	learn: 564.9246981	test: 469.8559787	best: 267.4440268 (36)	total: 1m 10s	remaining: 7.8s
180:	learn: 563.6582811	test: 470.1661712	best: 267.4440268 (36)	total: 1m 10s	remaining: 7.41s
181:	learn: 562.7647436	test: 469.7728851	best: 267.4440268 (36)	total: 1m 10s	remaining: 7.02s
182:	learn: 562.2589159	test: 469.2769882	best: 267.4440268 (36)	total: 1m 11s	remaining: 6.62s
183:	learn: 562.2535591	test: 469.1786585	best: 267.4440268 (36)	total: 1m 11s	remaining: 6.23s
184:	learn: 562.2497079	test: 469.1498264	best: 267.4440268 (36)	total: 1m 12s	remaining: 5.84s
185:	learn: 562.1859964	test: 469.0309633	bes

In [38]:
y_pred = cb_model.predict(X_test)

In [40]:
np.sqrt(mean_squared_error(y_pred, y_test))

275.1509529403856

In [41]:
np.sqrt(mean_squared_error(y_current_prediction, y_test))

2229.040538291336